In [112]:
from bs4 import BeautifulSoup

# # Correct file path
# file_path = r"nba_historic\nba_html_2019\quarter_data\q4\Joel Embiid_content_q4.html"

# # Opening the file with UTF-8 encoding
# with open(file_path, 'r', encoding='utf-8') as file:
#     html_content = file.read()

import os

def create_dataframe(relative_path, csv_path):

    # Specify the directory containing the files
    folder_path = relative_path      #r'nba_historic\nba_html_2019'

    # Loop through each file in the folder
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Check if it's a file (and not a subfolder)
        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding='utf-8') as file:
                # Read the content of the file
                html_content = file.read()
                original_filename = filename
                remove_string_in_filename = ".html"
                modified_filename = original_filename.replace(remove_string_in_filename,"") 
                print(f"Contents of {modified_filename}")

            # Parse the HTML content
            soup = BeautifulSoup(html_content, 'html.parser')

            # Find the table header row
            header_row = soup.find('tr', class_='Crom_headers__mzI_m')

            # Extract the text from each <th> element
            if header_row:
                headers = [th.text.strip() for th in header_row.find_all('th')]
                #print(headers)
            else:
                print("Header row not found.")

            rows = soup.find_all('tr')

            # Loop through rows and extract data
            list = []
            tbody = soup.find('tbody', class_='Crom_body__UYOcU')

            # Extract the rows and their data
            rows = tbody.find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                row_data = [cell.get_text(strip=True) for cell in cells]
                #print(row_data)
                list.append(row_data)
            #print(list)

            import pandas as pd 
            df = pd.DataFrame(list, columns=headers)
            #print(df.head(1))


            df[['Date', 'Matchup']] = df['Match Up'].str.split(' - ', expand=True)

            #splitting the match to get the team names
            df[['Team', 'Away']] = df['Matchup'].str.split(r' vs\. | @ ', expand=True)
            df[['Away_game', 'Home/Away_game']] = df['Matchup'].str.split(' @ ', expand=True)

            # Turns every none n/a value in this column into Away
            df[['Home/Away_game']] = df[['Home/Away_game']].map(lambda x: 'Away' if pd.notna(x) else x)
            # Turns every n/a value in this column into Home
            df[['Home/Away_game']] = df[['Home/Away_game']].fillna('Home')

            df['Date'] = pd.to_datetime(df['Date'])
            df['Matchup'] = df['Matchup'].astype('string')
            df['Team'] = df['Team'].astype('string')
            df['Away'] = df['Away'].astype('string')
            df['Home/Away_game'] = df['Home/Away_game'].astype('string')
            df['W/L'] = df['W/L'].astype('string')
            df['MIN'] = '00:' + df['MIN']
            df['MIN'] = pd.to_datetime(df['MIN'],format= '%H:%M:%S' ).dt.time

            columns_to_convert = [
                'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 
                'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', '+/-'
            ]

            # Converting selected columns to float
            df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')


            df = df[['Date', 
            'Matchup', 
            'Team',
            'Away',
            'Home/Away_game',
            'W/L', 
            'MIN', 
            'PTS', 
            'FGM', 
            'FGA',
            'FG%', 
            '3PM', 
            '3PA', 
            '3P%', 
            'FTM', 
            'FTA', 
            'FT%', 
            'OREB', 
            'DREB', 
            'REB', 
            'AST', 
            'STL', 
            'BLK', 
            'TOV', 
            'PF', 
            '+/-']]

            df_total_counts = df.isna().sum() + df.count()
            print(len(df))
            os.makedirs(csv_path, exist_ok=True)
            df.to_csv(f'nba_historic_csv\{csv_path}\{modified_filename}.csv', index=False)

if __name__ == "__main__":
    create_dataframe(r'nba_historic\nba_html_2019', r'nba_historic_csv\all_quarters')
    create_dataframe(r'nba_historic\nba_html_2019\quarter_data\q1', r'nba_historic_csv\quarter_data\q1' )
    create_dataframe(r'nba_historic\nba_html_2019\quarter_data\q2', r'nba_historic_csv\quarter_data\q2')
    create_dataframe(r'nba_historic\nba_html_2019\quarter_data\q3', r'nba_historic_csv\quarter_data\q3')
    create_dataframe(r'nba_historic\nba_html_2019\quarter_data\q4', r'nba_historic_csv\quarter_data\q4')






Contents of Andre Drummond_content
50


OSError: Cannot save file into a non-existent directory: 'nba_historic_csv\nba_historic_csv\all_quarters'

In [104]:
import pandas as pd 
df = pd.DataFrame(list, columns=headers)
df.head(50)

,Match Up,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,"Aug 09, 2020 - NOP vs. SAS",L,27:11,25,10,20,50.0,0,0,0.0,...,83.3,5,2,7,1,0,0,1,3,-21
1,"Aug 06, 2020 - NOP @ SAC",L,21:56,24,10,12,83.3,0,0,0.0,...,57.1,0,2,2,2,0,0,0,0,-12
2,"Aug 03, 2020 - NOP vs. MEM",W,25:07,23,9,21,42.9,0,1,0.0,...,50.0,3,4,7,5,0,0,2,2,8
3,"Aug 01, 2020 - NOP @ LAC",L,14:11,7,3,7,42.9,0,0,0.0,...,50.0,1,4,5,0,0,0,3,2,-17
4,"Jul 30, 2020 - NOP vs. UTA",L,15:10,13,6,8,75.0,0,0,0.0,...,100,0,0,0,1,0,0,2,3,-16
5,"Mar 08, 2020 - NOP @ MIN",W,31:42,23,9,15,60.0,1,1,100,...,66.7,1,6,7,1,0,1,3,0,20
6,"Mar 06, 2020 - NOP vs. MIA",W,32:04,17,7,9,77.8,0,0,0.0,...,75.0,2,4,6,2,1,2,3,1,1
7,"Mar 04, 2020 - NOP @ DAL",L,34:56,21,9,18,50.0,0,0,0.0,...,60.0,2,4,6,3,0,0,2,2,1
8,"Mar 03, 2020 - NOP vs. MIN",L,32:38,25,10,19,52.6,0,0,0.0,...,55.6,3,5,8,3,4,1,1,4,0
9,"Mar 01, 2020 - NOP vs. LAL",L,33:14,35,12,16,75.0,0,0,0.0,...,84.6,2,5,7,0,0,0,6,2,-10


In [105]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Match Up  24 non-null     object
 1   W/L       24 non-null     object
 2   MIN       24 non-null     object
 3   PTS       24 non-null     object
 4   FGM       24 non-null     object
 5   FGA       24 non-null     object
 6   FG%       24 non-null     object
 7   3PM       24 non-null     object
 8   3PA       24 non-null     object
 9   3P%       24 non-null     object
 10  FTM       24 non-null     object
 11  FTA       24 non-null     object
 12  FT%       24 non-null     object
 13  OREB      24 non-null     object
 14  DREB      24 non-null     object
 15  REB       24 non-null     object
 16  AST       24 non-null     object
 17  STL       24 non-null     object
 18  BLK       24 non-null     object
 19  TOV       24 non-null     object
 20  PF        24 non-null     object
 21  +/-       24 non-n

In [106]:
#splitting match up into date and team match up
df[['Date', 'Matchup']] = df['Match Up'].str.split(' - ', expand=True)

#splitting the match to get the team names
df[['Team', 'Away']] = df['Matchup'].str.split(r' vs\. | @ ', expand=True)
df[['Away_game', 'Home/Away_game']] = df['Matchup'].str.split(' @ ', expand=True)

# Turns every none n/a value in this column into Away
df[['Home/Away_game']] = df[['Home/Away_game']].map(lambda x: 'Away' if pd.notna(x) else x)
# Turns every n/a value in this column into Home
df[['Home/Away_game']] = df[['Home/Away_game']].fillna('Home')

df['Date'] = pd.to_datetime(df['Date'])
df['Matchup'] = df['Matchup'].astype('string')
df['Team'] = df['Team'].astype('string')
df['Away'] = df['Away'].astype('string')
df['Home/Away_game'] = df['Home/Away_game'].astype('string')
df['W/L'] = df['W/L'].astype('string')
df['MIN'] = '00:' + df['MIN']
df['MIN'] = pd.to_datetime(df['MIN'],format= '%H:%M:%S' ).dt.time

columns_to_convert = [
     'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 
    'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', '+/-'
]

# Converting selected columns to float
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

df = df[['Date', 
'Matchup', 
'Team',
'Away',
'Home/Away_game',
'W/L', 
'MIN', 
'PTS', 
'FGM', 
'FGA',
 'FG%', 
 '3PM', 
 '3PA', 
 '3P%', 
 'FTM', 
 'FTA', 
 'FT%', 
 'OREB', 
 'DREB', 
 'REB', 
 'AST', 
 'STL', 
 'BLK', 
 'TOV', 
 'PF', 
 '+/-']]



df.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            24 non-null     datetime64[ns]
 1   Matchup         24 non-null     string        
 2   Team            24 non-null     string        
 3   Away            24 non-null     string        
 4   Home/Away_game  24 non-null     string        
 5   W/L             24 non-null     string        
 6   MIN             24 non-null     object        
 7   PTS             24 non-null     int64         
 8   FGM             24 non-null     int64         
 9   FGA             24 non-null     int64         
 10  FG%             24 non-null     float64       
 11  3PM             24 non-null     int64         
 12  3PA             24 non-null     int64         
 13  3P%             24 non-null     float64       
 14  FTM             24 non-null     int64         
 15  FTA     

In [107]:

df = df[['Date', 'Matchup', 'Team','Away','Home/Away_game','W/L', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', '+/-']]
df

,Date,Matchup,Team,Away,Home/Away_game,W/L,MIN,PTS,FGM,FGA,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,2020-08-09,NOP vs. SAS,NOP,SAS,Home,L,00:27:11,25,10,20,...,83.3,5,2,7,1,0,0,1,3,-21
1,2020-08-06,NOP @ SAC,NOP,SAC,Away,L,00:21:56,24,10,12,...,57.1,0,2,2,2,0,0,0,0,-12
2,2020-08-03,NOP vs. MEM,NOP,MEM,Home,W,00:25:07,23,9,21,...,50.0,3,4,7,5,0,0,2,2,8
3,2020-08-01,NOP @ LAC,NOP,LAC,Away,L,00:14:11,7,3,7,...,50.0,1,4,5,0,0,0,3,2,-17
4,2020-07-30,NOP vs. UTA,NOP,UTA,Home,L,00:15:10,13,6,8,...,100.0,0,0,0,1,0,0,2,3,-16
5,2020-03-08,NOP @ MIN,NOP,MIN,Away,W,00:31:42,23,9,15,...,66.7,1,6,7,1,0,1,3,0,20
6,2020-03-06,NOP vs. MIA,NOP,MIA,Home,W,00:32:04,17,7,9,...,75.0,2,4,6,2,1,2,3,1,1
7,2020-03-04,NOP @ DAL,NOP,DAL,Away,L,00:34:56,21,9,18,...,60.0,2,4,6,3,0,0,2,2,1
8,2020-03-03,NOP vs. MIN,NOP,MIN,Home,L,00:32:38,25,10,19,...,55.6,3,5,8,3,4,1,1,4,0
9,2020-03-01,NOP vs. LAL,NOP,LAL,Home,L,00:33:14,35,12,16,...,84.6,2,5,7,0,0,0,6,2,-10


In [108]:

df.Date = pd.to_datetime(df.Date)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            24 non-null     datetime64[ns]
 1   Matchup         24 non-null     string        
 2   Team            24 non-null     string        
 3   Away            24 non-null     string        
 4   Home/Away_game  24 non-null     string        
 5   W/L             24 non-null     string        
 6   MIN             24 non-null     object        
 7   PTS             24 non-null     int64         
 8   FGM             24 non-null     int64         
 9   FGA             24 non-null     int64         
 10  FG%             24 non-null     float64       
 11  3PM             24 non-null     int64         
 12  3PA             24 non-null     int64         
 13  3P%             24 non-null     float64       
 14  FTM             24 non-null     int64         
 15  FTA     